In [4]:
# create and solve basic equilibrium.
# taking Example_1 and putting it all here to use as eq for testing probes


import os
os.environ["ACTIVE_COILS_PATH"] = "../machine_configs/MAST-U/active_coils_from_UDA.pickle"
os.environ["PASSIVE_COILS_PATH"] = "../machine_configs/MAST-U/passive_coils.pickle"
os.environ["WALL_PATH"] = "../machine_configs/MAST-U/wall.pickle"
os.environ["LIMITER_PATH"] = "../machine_configs/MAST-U/limiter.pickle"
# the following environment variable is optional, 
# it contains data on an equilibrium that is representative for the tokamak 
# for more info please refer to Example_0
os.environ["EQUILIBRIUM_PATH"] = "../machine_configs/MAST-U/simple_initilization_equilibrium.pickle"

# Now the machine can actually be built:
from freegsnke import build_machine
tokamak = build_machine.tokamak()

from freegsnke import equilibrium_update
from freegsnke.jtor_update import ConstrainPaxisIp
from freegsnke import GSstaticsolver



# create equilibirum object
eq = equilibrium_update.Equilibrium(
    tokamak=tokamak,
    Rmin=0.1, Rmax=2.0,   # Radial range
    Zmin=-2.2, Zmax=2.2,  # Vertical range
    nx=65,   # Number of grid points in the radial direction
    ny=129,  # Number of grid points in the vertical direction
    # note that both nx and ny have to be of the form 2**i + 1 with i being an integer
    # psi=plasma_psi
)  


# create profile object
profiles = ConstrainPaxisIp(
    eq=eq,
    limiter=tokamak.limiter,
    paxis=8e3,
    Ip=6e5,
    fvac=0.5,
    alpha_m=1.8,
    alpha_n=1.2
)

# load nonlinear solver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq)

# setup initial conditions....
# Set X-point coordinates
Rx = 0.6
Zx = 1.1
xpoints = [(Rx, -Zx),   
           (Rx,  Zx)]

# Set any desired isoflux constraints,
# these have structure (R1, Z1, R2, Z2)
# where (R1, Z1) and (R2, Z2) are 
# desired to be on the same flux contour.
Rmid = 1.4   # Outboard midplane
Rin = 0.35    # Inboard midplane
isoflux = [(Rx,Zx, Rx,-Zx), # link X-points
           (Rmid, 0, Rin, 0.0), # link inner and outer midplane points
           (Rmid, 0, Rx, Zx), # link outer midplane point and X-point

        #   some additional constraints
           (Rmid,0, 1.2,.7),
           (Rmid,0, 1.2,-.7),
           (Rx, Zx, .85, 1.7),
           (Rx, Zx, .75, 1.6),
           (Rx, Zx, Rin, 0.2),
           (Rx, Zx, Rin, 0.1),
           (Rx,-Zx, Rin, -0.1),
           (Rx,-Zx, Rin, -0.2),
           (Rx,-Zx, .85, -1.7),
           (Rx,-Zx, .75, -1.6),
           (Rx,-Zx, 0.45, -1.8),
           (Rx, Zx, 0.45,  1.8),
           ]
           
# Instantiate the constrain object
constrain = freegs.control.constrain(xpoints=xpoints, 
                                     isoflux=isoflux,
                                     gamma=5e-7
                                    )

# apply solver
GSStaticSolver.solve(eq=eq, 
                     profiles=profiles, 
                     constrain=constrain, 
                     target_relative_tolerance=1e-3)

Equilibrium initialised using file provided as part of the machine description.


AttributeError: 'super' object has no attribute 'Jtor_part1'

This is only the inverse solve, but will have coil and plasma currents that can be used by the probes.

Let's set this up now.

In [2]:
from freegsnke import magnetic_probes

flux_probe = magnetic_probes.probe(eq)

ValueError: PASSIVE_COILS_PATH environment variable not set.